# Setup

## Instalações e Importações

In [1]:
#instalar a biblioteca MySQL Connector para o Python.
#!pip install mysql-connector-python

In [2]:
#import das bibliotecas mysql.connector e pandas.
import mysql.connector
from mysql.connector import Error
import pandas as pd

obs: a função Error foi importada separadamente para facilitar o acesso a ela, caso contrário seria necessário fazer a chamada da função como 'mysql.connector.Error'.

## Conectando ao MySQL Server

In [3]:
#Vamos definir uma função de criação da conexão com o db para facilitar a chamada 
#caso seja necessário conectar com mais de um db.
  
def create_db_connection(host_name, user_name, user_password, db_name):
    conexao = None
    try:
        conexao = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print('Conexão Realizada com sucesso!')
    except Error as err:
        print(f"Erro ao conectar com o Banco: '{err}'")

    return conexao

Compartilhando aqui um pouco da experiência que tive com essa conexão com o Mysql Server:

- Criei o banco de dados na minha máquina utilizando o xampp ('XAMPP é um pacote com os principais servidores de código aberto do mercado, incluindo FTP, banco de dados MySQL e Apache com suporte as linguagens PHP e Perl.' by wikipedia).

- Para criar o banco de dados, utilizei a página de gerenciamento do xampp (http://localhost/phpmyadmin/), criei o db crédito e importei o arquivo credito.sql para criação da tabela e inserção dos registros.

- As vezes é preciso 'matar' o apache e o mysql no terminal (uso a dist linux debian buster 11) e reiniciar o xampp no painel de controle para acessar a página de gerenciamento. Quando necessário, utilizo os seguintes comandos:
    #sudo apachectl stop
    #sudo service mysql stop

- Alerto que não consegui conectar com o localhost utilizando o google colab, somente rodando o notebook jupyter dentro da minha máquina através do anaconda

In [4]:
#Criar conexão. Nao esquecer de inserir os valores para os parametros 
#da conexão (host, user, password e database).

conexao = create_db_connection(host_name='localhost', 
                               user_name='user', 
                               user_password='password', 
                               db_name='crédito')

Conexão Realizada com sucesso!


In [5]:
#Em execuções anteriores identifiquei um problema recorrente que era a perda da conexão
#com o db após 3 ou 4 consultas, para contornar essa situação vamos definir uma função
#que checa se a conexão está ativa e caso contrário restabelece a conexão.

def check_db_connection(conexao):
    try:
        conexao.cursor()
    except:
        conexao.reconnect()

# Consultas

Para aplicar os comandos SQL ao banco de dados da nossa conexão é preciso instaciar o objeto cursor #cursor = conexao.cursor(), é através desse objeto que iremos conseguir executar as queries com a função execute #cursor.execute(query)

In [6]:
#Como a criação do cursor e execução da consulta são ações recorrentes para cada consulta,
#iremos utilizar uma função e assim evitar a repetição do código.

def read_query(conexao, query):
    check_db_connection(conexao) 
    cursor = conexao.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()    
    except Error as err:
        print(f"Error: '{err}'")
    return result

## Q1: Mostrar o tipo das colunas da tabela credito.

In [7]:
q1 = """
DESCRIBE creditos;
"""

#Executar consulta.
retorno = read_query(conexao, q1)

#Formatando o resultado em um DataFrame do Pandas.
columns = ['field','type','null','key','default','extra']
df = pd.DataFrame(retorno, columns=columns)
display(df)

,field,type,null,key,default,extra
0,id,int(11),NO,PRI,None,
1,padrao,int(11),NO,,None,
2,idade,int(11),NO,,None,
3,sexo,varchar(1),NO,,None,
4,dependentes,int(11),NO,,None,
5,escolaridade,varchar(19),NO,,None,
6,estado_civil,varchar(10),NO,,None,
7,salario_anual,varchar(14),NO,,None,
8,tipo_cartao,varchar(8),NO,,None,
9,meses_de_relacionamento,int(11),NO,,None,


## Q2: Mostrar o total de registros da tabela credito.

In [8]:
q2 = """
SELECT COUNT(*) FROM creditos;
"""

retorno = read_query(conexao, q2)

for item in retorno:
    print(item[0])

10127


## Q3: Mostrar apenas as primeiras 20 linhas tabela credito.

In [9]:
q3 = """
SELECT * FROM creditos LIMIT 20;
"""

retorno = read_query(conexao, q3)

#As consultas retornam apenas os registros, deixando o nome das colunas de fora.
for item in retorno:
    print(item)

(708082083, 0, 45, 'F', 3, 'ensino medio', 'casado', 'menos que $40K', 'blue', 36, 4, 3, 3, '3.544,02', '15.149,31', 111)
(708083283, 1, 58, 'M', 0, 'na', 'solteiro', '$40K - $60K', 'blue', 45, 3, 3, 1, '3.421,03', '992,47', 21)
(708084558, 1, 46, 'M', 3, 'doutorado', 'divorciado', '$80K - $120K', 'blue', 38, 6, 3, 3, '8.258,61', '1.447,53', 23)
(708085458, 0, 34, 'F', 2, 'sem educacao formal', 'solteiro', 'menos que $40K', 'blue', 24, 6, 2, 2, '1.438,55', '3.940,59', 82)
(708086958, 0, 49, 'F', 2, 'sem educacao formal', 'casado', 'na', 'blue', 41, 3, 2, 5, '3.128,46', '4.369,86', 59)
(708095133, 0, 43, 'M', 4, 'na', 'na', '$120K +', 'blue', 34, 5, 2, 2, '33.304,71', '1.448,54', 29)
(708098133, 0, 32, 'F', 0, 'mestrado', 'casado', 'menos que $40K', 'blue', 19, 6, 0, 1, '2.834,54', '1.598,37', 39)
(708099183, 0, 37, 'F', 2, 'ensino medio', 'solteiro', 'menos que $40K', 'blue', 36, 4, 2, 2, '5.723,85', '2.732,91', 63)
(708100533, 0, 55, 'F', 3, 'graduacao', 'solteiro', 'menos que $40K', 

In [10]:
#para obter o nome das colunas utilizaremos também uma query:
q3_columns = """ 
SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'creditos';")
"""
columns_table = read_query(conexao, q3_columns)

#q3_columns retorna uma lista de tuplas = [(id,),(padrao,),(idade,),(sexo,),...,(qtd_transacoes_12m,)].

#para o cabecalho do nosso df queremos apenas o primeiro elemento de cada tupla (id,) = id.
coluns_df = []
for item in columns_table:
    coluns_df.append(item[0])

In [11]:
#Formatando o resultado em um DataFrame do Pandas.
df = pd.DataFrame(retorno, columns=coluns_df)
display(df)

,id,padrao,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,708082083,0,45,F,3,ensino medio,casado,menos que $40K,blue,36,4,3,3,"3.544,02","15.149,31",111
1,708083283,1,58,M,0,na,solteiro,$40K - $60K,blue,45,3,3,1,"3.421,03","992,47",21
2,708084558,1,46,M,3,doutorado,divorciado,$80K - $120K,blue,38,6,3,3,"8.258,61","1.447,53",23
3,708085458,0,34,F,2,sem educacao formal,solteiro,menos que $40K,blue,24,6,2,2,"1.438,55","3.940,59",82
4,708086958,0,49,F,2,sem educacao formal,casado,na,blue,41,3,2,5,"3.128,46","4.369,86",59
5,708095133,0,43,M,4,na,na,$120K +,blue,34,5,2,2,"33.304,71","1.448,54",29
6,708098133,0,32,F,0,mestrado,casado,menos que $40K,blue,19,6,0,1,"2.834,54","1.598,37",39
7,708099183,0,37,F,2,ensino medio,solteiro,menos que $40K,blue,36,4,2,2,"5.723,85","2.732,91",63
8,708100533,0,55,F,3,graduacao,solteiro,menos que $40K,blue,36,3,3,3,"2.679,04","4.943,89",85
9,708103608,0,52,M,1,ensino medio,solteiro,$60K - $80K,blue,45,1,1,5,"11.898,27","15.798,92",128


## Q4: Mostrar a quantidade de pessoas por nível de escolaridade.

In [12]:
q4 = """
SELECT escolaridade, COUNT(*) AS qtd 
FROM creditos 
GROUP BY escolaridade
ORDER BY 2 DESC;
"""

retorno = read_query(conexao, q4)

df = pd.DataFrame(retorno, columns=['escolaridade','qtd'])
display(df)

,escolaridade,qtd
0,mestrado,3128
1,ensino medio,2013
2,na,1519
3,sem educacao formal,1487
4,graduacao,1013
5,doutorado,967


## Q5: Mostrar a quantidade de pessoas por estado civil.

In [13]:
q5 = """
SELECT estado_civil, COUNT(*) AS qtd 
FROM creditos 
GROUP BY estado_civil
ORDER BY 2 DESC;
"""

retorno = read_query(conexao, q5)

df = pd.DataFrame(retorno, columns=['estado_civil','qtd'])
display(df)

,estado_civil,qtd
0,casado,4687
1,solteiro,3943
2,na,749
3,divorciado,748


## Q6: Mostrar a quantidade de pessoas por faixa salarial. 

In [14]:
q6 = """
SELECT salario_anual, COUNT(*) AS qtd,
	CASE
		WHEN salario_anual = 'na' THEN 1
		WHEN salario_anual = 'Menos que $40K' THEN 2
		WHEN salario_anual = '$40K - $60K' THEN 3
		WHEN salario_anual = '$60K - $80K' THEN 4
		WHEN salario_anual = '$80K - $120K' THEN 5
		ELSE 6
	END AS ordem
FROM creditos 
GROUP BY salario_anual
ORDER BY ordem;
"""

retorno = read_query(conexao, q6)

df = pd.DataFrame(retorno,columns=['salario_anual','qtd','ordem'])
display(df)

,salario_anual,qtd,ordem
0,na,1112,1
1,menos que $40K,3561,2
2,$40K - $60K,1790,3
3,$60K - $80K,1402,4
4,$80K - $120K,1535,5
5,$120K +,727,6


## Q7: Mostrar os tipos de cartão da tabela credito.

In [15]:
q7 = """
SELECT DISTINCT tipo_cartao 
FROM creditos;
"""

retorno = read_query(conexao, q7)

for item in retorno:
    print(item[0])

blue
silver
gold
platinum


## Q8: Mostrar a quantidade de gêneros existentes nessa tabela.

In [16]:
q8 = """
SELECT COUNT(DISTINCT sexo)
FROM creditos;
"""

retorno = read_query(conexao, q8)

for item in retorno:
    print(item[0])

2


## Q9: Mostrar a quantidade de pessoas por gênero. 

In [17]:
q9 = """
SELECT sexo, COUNT(*) AS qtd 
FROM creditos 
GROUP BY sexo
ORDER BY 2 DESC;
"""

retorno = read_query(conexao, q9)

df = pd.DataFrame(retorno, columns=['sexo','qtd'])
display(df)

,sexo,qtd
0,F,5358
1,M,4769


## Q10: Verificar a quantidade de pessoas (porcentagem) em relação aos outros níveis de escolaridade. 

In [18]:
q10 = """
SELECT escolaridade, COUNT(*)/(SELECT COUNT(*) FROM creditos) * 100 AS taxa
FROM creditos
GROUP BY escolaridade;
"""

retorno = read_query(conexao, q10)

df = pd.DataFrame(retorno, columns=['escolaridade','taxa'])
df['taxa'] = df['taxa'].apply(lambda x: round(x, 2))
display(df)

,escolaridade,taxa
0,doutorado,9.55
1,ensino medio,19.88
2,graduacao,10.00
3,mestrado,30.89
4,na,15.00
5,sem educacao formal,14.68


## Q11: Quantos grupos de pessoas com mestrado podemos ter usando escolaridade, estado civil como filtros? 

In [19]:
q11 = """
SELECT estado_civil, COUNT(*)
FROM creditos 
WHERE escolaridade = 'mestrado'
GROUP BY estado_civil;
"""

retorno = read_query(conexao, q11)

df = pd.DataFrame(retorno, columns=['estado_civil','qtd'])
display(df)

,estado_civil,qtd
0,casado,1479
1,divorciado,225
2,na,227
3,solteiro,1197


## Q12: Qual o total de pessoas com escolaridade mestrado e estado civil casado? 

In [20]:
q12 = """
SELECT COUNT(*) 
FROM creditos 
WHERE escolaridade = 'mestrado' AND estado_civil='casado';
"""

retorno = read_query(conexao, q12)

for item in retorno:
    print(item[0])

1479


## Q13: Qual cartão essas pessoas do exercício anterior possuem? 

In [21]:
q13 = """
SELECT tipo_cartao, COUNT(*)
FROM creditos 
WHERE escolaridade = 'mestrado' AND estado_civil='casado'
GROUP BY tipo_cartao;
"""

retorno = read_query(conexao, q13)

df = pd.DataFrame(retorno, columns=['tipo_cartao','qtd'])
display(df)

,tipo_cartao,qtd
0,blue,1391
1,gold,12
2,platinum,1
3,silver,75


## Q14: Quantos grupos de pessoas com mestrado podemos ter usando escolaridade, estado civil e salário anual como filtros? 

In [22]:
q14 = """
SELECT escolaridade, estado_civil, salario_anual, COUNT(*)
FROM creditos
WHERE escolaridade = 'mestrado'
GROUP BY escolaridade, estado_civil, salario_anual;
"""

retorno = read_query(conexao, q14)

df = pd.DataFrame(retorno, columns=['escolaridade', 'estado_civil', 'salario_anual','qtd'])
display(df)

,escolaridade,estado_civil,salario_anual,qtd
0,mestrado,casado,$120K +,103
1,mestrado,casado,$40K - $60K,251
2,mestrado,casado,$60K - $80K,205
3,mestrado,casado,$80K - $120K,239
4,mestrado,casado,menos que $40K,528
5,mestrado,casado,na,153
6,mestrado,divorciado,$120K +,11
7,mestrado,divorciado,$40K - $60K,41
8,mestrado,divorciado,$60K - $80K,35
9,mestrado,divorciado,$80K - $120K,32


## Q15: Mostrar o total de clientes da base. 

In [23]:
q15 = """
SELECT (COUNT(id) + SUM(dependentes)) AS clientes_cadastrados FROM creditos;
"""

retorno = read_query(conexao, q15)

for item in retorno:
    print(item[0])

33887


## Q16: Mostrar quantas pessoas deixaram de responder alguma informação sobre escolaridade, estado civil ou Salario Anual (Considerar o nulo como 'na') e quantas responderam. 

In [24]:
q16 = """
SELECT
(SELECT COUNT(*)
FROM creditos
WHERE escolaridade = 'na' OR estado_civil = 'na' OR salario_anual = 'na') AS na,
COUNT(*) - (SELECT COUNT(*)
FROM creditos
WHERE escolaridade = 'na' OR estado_civil = 'na' OR salario_anual = 'na') AS validos
FROM creditos; 
"""

retorno = read_query(conexao, q16)

df = pd.DataFrame(retorno, columns=['nao_respondido','respondido'])
display(df)

,nao_respondido,respondido
0,3046,7081


## Q17: Quantos clientes não informaram o salario anual? 

In [25]:
q17 = """
SELECT COUNT(*)
FROM creditos
WHERE salario_anual = 'na';
"""

retorno = read_query(conexao, q17)

for item in retorno:
    print(item[0])

1112


## Q18: Qual é o cartão das pessoas que não informaram salario anual na base? 

In [26]:
q18 = """
SELECT DISTINCT tipo_cartao
FROM creditos
WHERE salario_anual = 'na';
"""

retorno = read_query(conexao, q18)

for item in retorno:
    print(item[0])

blue
silver
gold
platinum


## Q19: Qual é o menor e o maior limite de crédito das pessoas que não informaram salario anual na base?

In [27]:
q19 = """
SELECT MIN(limite_credito) AS Menor_limite, MAX(limite_credito) AS Maior_limite
FROM creditos;
"""

retorno = read_query(conexao, q19)

df = pd.DataFrame(retorno, columns=['minimo','maximo'])
display(df)

,minimo,maximo
0,"1.438,00","9.995,38"


## Q20: Qual é a quantidade de pessoas que não declararam nenhuma dessas três informações (escolaridade, estado civil e salário anual) contida nesta base? 

In [28]:
q20 = """
SELECT COUNT(*)
FROM creditos
WHERE escolaridade = 'na' AND estado_civil = 'na' AND salario_anual = 'na';
"""

retorno = read_query(conexao, q20)

for item in retorno:
    print(item[0])

7


## Q21: Sobre essas pessoas que possuem cartão e que não declararam essas informações, quanto de limite de crédito tem cada uma? 

In [29]:
q21 = """
SELECT id, limite_credito
FROM creditos
WHERE escolaridade = 'na' AND estado_civil = 'na' AND salario_anual = 'na';
"""

retorno = read_query(conexao, q21)

df = pd.DataFrame(retorno, columns=['id','limite_credito'])
display(df)

,id,limite_credito
0,709375158,"7.270,56"
1,712019883,"12.778,47"
2,714127383,"4.431,84"
3,715007583,"1.801,66"
4,719329983,"15.648,25"
5,755959683,"3.410,15"
6,769701258,"12.463,36"
